# OZON Parser.PY

In [5]:
# 
import time
import json

from selenium import webdriver

# Модуль для подмены параметров пользователя
from selenium_stealth import stealth
from bs4 import BeautifulSoup

# 
from curl_cffi import requests

In [4]:
# !pip install selenium_stealth
# !pip install curl_cffi

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.3 MB 932.9 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 1.0 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 1.1 MB/s eta 0:00:00


##### ChromeDriver
The latest drivers can be found at:

https://sites.google.com/chromium.org/driver

In [7]:
# Простой код для имитации входа на сайт

# from selenium import webdriver
# import time

driver = webdriver.Chrome()
driver.get('https://oson.ru')
time.sleep(10)

# Не удается получить доступ к сайту
# Превышено время ожидания ответа от сайта oson.ru.

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=132.0.6834.197)
Stacktrace:
	GetHandleVerifier [0x00007FF6778102F5+28725]
	(No symbol) [0x00007FF677772AE0]
	(No symbol) [0x00007FF67760510A]
	(No symbol) [0x00007FF677602861]
	(No symbol) [0x00007FF6775F3559]
	(No symbol) [0x00007FF6775F52AF]
	(No symbol) [0x00007FF6775F381F]
	(No symbol) [0x00007FF6775F32ED]
	(No symbol) [0x00007FF6775F2FBA]
	(No symbol) [0x00007FF6775F0DD1]
	(No symbol) [0x00007FF6775F145C]
	(No symbol) [0x00007FF67760824A]
	(No symbol) [0x00007FF6776A0FAE]
	(No symbol) [0x00007FF67767FFAA]
	(No symbol) [0x00007FF6776A0181]
	(No symbol) [0x00007FF67767FD53]
	(No symbol) [0x00007FF67764A0E3]
	(No symbol) [0x00007FF67764B471]
	GetHandleVerifier [0x00007FF677B3F30D+3366989]
	GetHandleVerifier [0x00007FF677B512F0+3440688]
	GetHandleVerifier [0x00007FF677B478FD+3401277]
	GetHandleVerifier [0x00007FF6778DAAAB+858091]
	(No symbol) [0x00007FF67777E74F]
	(No symbol) [0x00007FF67

In [11]:
# Функция для обхода защиты от скрапинга - # Модуль для подмены параметров браузера

def init_webdriver():
    driver = webdriver.Chrome()
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)
    driver.maximize_window()
    return driver

In [9]:
# init_webdriver()

<selenium.webdriver.chrome.webdriver.WebDriver (session="09efe2c32e9ae17c38ca05ae40d3c63c")>

In [13]:
# Если после запуска этого модуля попасть на сайт, то:
driver = init_webdriver()
# driver = webdriver.Chrome()
# driver.get('https://oson.ru')
driver.get('https://www.ozon.ru/?__rr=1&abt_att=1&origin_referer=www.google.com')
time.sleep(10)

In [ ]:
# Функция для прокрутки страницы
def scrolldown(driver, deep):
    for _ in range(deep):
        driver.execute_script('window.scrollBy(0, 500)') # 0 - перемещение по X ; 500 - перемещение по Y
        time.sleep(0.1)

In [ ]:
# 

def get_product_info(product_url):
    session = requests.Session()

    raw_data = session.get("https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=" + product_url)
# https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=product/pandora-sharm-1653191574/?at=99trjPgn0sGVJG1jfABrrLJuLXP2YQtQjZklGI1jqWP8
# https://www.ozon.ru/
    json_data = json.loads(raw_data.content.decode())

    full_name = json_data["seo"]["title"]

    if json_data["layout"][0]["component"] == "userAdultModal":
        product_id = str(full_name.split()[-1])[1:-1]
        print(product_id, full_name)
        return (product_id, full_name, "Товар для лиц старше 18 лет", None, None)
    else:
        description = json.loads(json_data["seo"]["script"][0]["innerHTML"])["description"]
        image_url = json.loads(json_data["seo"]["script"][0]["innerHTML"])["image"]
        price = json.loads(json_data["seo"]["script"][0]["innerHTML"])["offers"]["price"] + " " +\
                json.loads(json_data["seo"]["script"][0]["innerHTML"])["offers"]["priceCurrency"]
        rating = json.loads(json_data["seo"]["script"][0]["innerHTML"]["ratingValue"])
        rating_counter = json.loads(json_data["seo"]["script"][0]["innerHTML"]["reviewCount"])
        product_id = json.loads(json_data["seo"]["script"][0]["innerHTML"])["sku"]

        return (product_id, full_name, description, price, rating, rating_counter, image_url)

In [ ]:
# Функция для определения блоков с информацией

def get_mainpage_cards(driver, url):
    driver.get(url)
    scrolldown(driver, 50)
    main_page_html = BeautifulSoup(driver.page_source, "html.parser")

# Это главный тэг в котором находятся все карточки
    content = main_page_html.find("div", {"class": "container"})

# Собираем все вложенные тэги ("recursive=False" не позволяет искать вложенные тэги), то есть на 1 уровень вложенности и забираем ПАГИНАТОР
    content = content.findChildren(recursive=False)[-1].find("div")

# Углубляемся в ПАГИНАТОР и забираем все ДИВы
    content = content.findChildren(recursive=False)

# Собираем список из тэгов, содержащих "freshIsland" и забираем из него самый последний ИЛИ ПРОСТО "island"
    content = [item for item in content if "freshIsland" in str(item)][-1]

# Переходим последовательно внутрь блоков: 1)оболочка; 2)пагинатор; 3)блок с классами - третий вложенный блок
    content = content.find("div").find("div").find("div")

# Во вложенном блоке лежит секция с карточками товаров (это опять блок - оболочка в котором лежат карточки товаров) !!!!!!
    content = content.findChildren(recursive=False)

# Теперь !!!!! НАКОНЕЦ-ТО !!!!! можно пройти циклом по карточкам товаров, лежащим в одной (первой секции)
    all_cards = list() # список для всех карточек во всех слоях
    for layer in content:
        layer = layer.find("div") # это блок с карточкой, который в свою очередь состоит из 3-х блоков
        cards = layer.findChildren(recursive=False) # забираем все дочерние блоки ()

        cards_in_layer = list() # список для карточек в КАЖДОМ ИЗ слоёв
        for card in cards:
            card = card.findChildren(recursive=False) # добираемся до блока с информацией - это последний из дочерних блоков

            card_name = card[2].find("span", {"class": "tsBody500Medium"}).contents[0] # это всё текстовое содержимое блока (tsBody500Medium)
            card_url = card[2].find("a", href=True)["href"] # относительная ссылка на карточку
            product_url = "https://ozon.ru/" + card_url # абсолютная ссылка на карточку

            product_id, full_name, description, price, rating, rating_counter, image_url = get_product_info(card_url)
            card_info = {product_id: {"short_name": card_name,
                                      "full_name": full_name,
                                      "description": description,
                                      "url": product_url,
                                      "rating": rating,
                                      "rating_counter": rating_counter,
                                      "price": price,
                                      "image_url": image_url
                                      }
                         }
            cards_in_layer.append(card_info)
            print(product_id, "- DONE")

        all_cards.extend(cards_in_layer)
    return all_cards

In [ ]:
# 

def get_searchpage_cards(driver, url, all_cards = []):
    driver.get(url)
    scrolldown(driver, 20)
    search_page_html = BeautifulSoup(driver.page_source, "html.parser")

    content = search_page_html.find("div", {"id": "layoutPage"})
    content = content.find("div")

    content_with_cards = content.find("div", {"class": "widget-search-result-container"})
    content_with_cards = content_with_cards.find("div").findChildren(recursive=False)

    cards_in_page = list()
    for card in content_with_cards:
        card_url = card.find("a", href=True)["href"]
        card_name = card.find("span", {"class": "tsBody500Medium"}).contents[0]

        product_url = "https://ozon.ru/" + card_url

        product_id, full_name, description, price, rating, rating_counter, image_url = get_product_info(card_url)
        card_info = {product_id: {"short_name": card_name,
                                  "full_name": full_name,
                                  "description": description,
                                  "url": product_url,
                                  "rating": rating,
                                  "rating_counter": rating_counter,
                                  "price": price,
                                  "image_url": image_url
                                  }
                     }
        cards_in_page.append(card_info)
        print(product_id, "- DONE")

    content_with_next = [div for div in content.find_all("a", href=True) if "Дальше" in str(div)]
    if not content_with_next:
        return cards_in_page
    else:
        next_page_url = "https://www.ozon.ru" + content_with_next[0]["href"]
        all_cards.extend(get_searchpage_cards(driver, next_page_url, cards_in_page))
        return all_cards

In [ ]:
# 

if __name__ == "__main__":
    url_ozon = "https://www.ozon.ru"

    driver = init_webdriver()

    search_list = ["пиво+светлое", "бандана+мужская+с+черепами", "сухарики+кириешки", "RTX+4090"]
    end_list = list()

    try:
        main_cards = get_mainpage_cards(driver, url_ozon)
        print("Я успешно нашёл", len(main_cards), "на главной странице")
        end_list.append("MAIN")
    except:
        print("Я упал на парсинге главной страницы")

    for search_tag in search_list:
        url_search = f"https://www.ozon.ru/search/?text={search_tag}&from_global=true"

        try:
            search_cards = get_searchpage_cards(driver, url_search)
            print("Я успешно нашёл", len(search_cards), "по поиску", search_tag)
            end_list.append(search_tag)
        except:
            print("Я упал на", search_tag)
    print(end_list)


# GET PRODUCT INFO.PY

In [ ]:
# 

from curl_cffi import requests
import json

In [ ]:
# формируем сессию и получаем куки

s = requests.Session()
s.get("https://www.ozon.ru")
print(*str(s.cookies).split(", "), sep="\n")

url_prod = "https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/artbuk-h-r-a-m-monohromnyy-uzhas-h-r-a-m-1173897628/?avtc=1&avte=2&avts=1719315501"

r = s.get("https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/futbolka-rustactic-politsiya-991026398")

#print(r)
json_data = json.loads(r.content.decode())
print(json_data["seo"]["title"])
print(json.loads(json_data["seo"]["script"][0]["innerHTML"])["description"])
print(json.loads(json_data["seo"]["script"][0]["innerHTML"])["image"])
print(json.loads(json_data["seo"]["script"][0]["innerHTML"])["offers"]["price"] + " " +
      json.loads(json_data["seo"]["script"][0]["innerHTML"])["offers"]["priceCurrency"])
print(json.loads(json_data["seo"]["script"][0]["innerHTML"])["sku"])